In [2]:
# Body Fat Prediction using ANN (Artificial Neural Network) - Pure ANN Implementation
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

class ANNBodyFatPredictor:
    def __init__(self):
        self.ann_model = None  # ANN model
        self.scaler = StandardScaler()
        self.feature_columns = ['Age', 'Weight', 'Height', 'Neck', 'Chest', 'Abdomen',
                               'Hip', 'Thigh', 'Knee', 'Ankle', 'Biceps', 'Forearm', 'Wrist']
        self.target_column = 'BodyFat'
        self.is_trained = False

    def create_structured_dataset(self, n_samples=1500):
        """Tạo dữ liệu có cấu trúc dựa trên công thức khoa học (không có yếu tố ngẫu nhiên)"""

        data_list = []

        # Định nghĩa các nhóm tuổi dựa trên thống kê y khoa
        age_groups = [
            {'min_age': 18, 'max_age': 29, 'count': 300, 'avg_weight': 72, 'avg_height': 175, 'base_bf': 12},
            {'min_age': 30, 'max_age': 39, 'count': 350, 'avg_weight': 76, 'avg_height': 174, 'base_bf': 16},
            {'min_age': 40, 'max_age': 49, 'count': 400, 'avg_weight': 80, 'avg_height': 173, 'base_bf': 20},
            {'min_age': 50, 'max_age': 59, 'count': 300, 'avg_weight': 83, 'avg_height': 172, 'base_bf': 24},
            {'min_age': 60, 'max_age': 70, 'count': 150, 'avg_weight': 78, 'avg_height': 171, 'base_bf': 27}
        ]

        # Tạo dữ liệu có cấu trúc cho từng nhóm tuổi (pattern-based, không phải ngẫu nhiên)
        for group in age_groups:
            for i in range(group['count']):
                # Tạo age theo pattern tuyến tính
                age_range = group['max_age'] - group['min_age']
                age = group['min_age'] + (i / group['count']) * age_range
                age = int(max(group['min_age'], min(group['max_age'], age)))

                # Tạo height theo pattern có cấu trúc
                height_pattern = (i % 20) - 10  # Pattern từ -10 đến +10
                height = group['avg_height'] + height_pattern * 0.8
                height = max(155, min(195, height))

                # Tạo weight dựa trên BMI categories xác định
                bmi_categories = [18.5, 21.0, 23.5, 26.0, 29.0, 33.0]  # BMI values
                bmi_idx = i % len(bmi_categories)
                target_bmi = bmi_categories[bmi_idx]
                weight = target_bmi * (height/100)**2

                # Điều chỉnh weight theo pattern
                weight_adjustment = (i % 10) - 5  # Pattern từ -5 đến +5
                weight = max(45, min(140, weight + weight_adjustment))

                # Tính body fat dựa trên công thức khoa học (Deurenberg equation)
                bmi = weight / (height/100)**2

                # Công thức Deurenberg: BF% = (1.20 × BMI) + (0.23 × Age) - (10.8 × Gender) - 5.4
                # Với Gender = 1 cho nam giới
                base_bf = (1.20 * bmi) + (0.23 * age) - (10.8 * 1) - 5.4

                # Điều chỉnh dựa trên pattern
                pattern_adjustment = (i % 8) - 4  # Pattern từ -4 đến +4
                body_fat = base_bf + pattern_adjustment
                body_fat = max(4, min(45, body_fat))

                # Tính toán các measurements dựa trên công thức anthropometric chuẩn

                # Neck circumference - công thức dựa trên nghiên cứu khoa học
                neck = 37.5 + (weight - 70) * 0.10 + (height - 170) * 0.035
                neck_pattern = (i % 6) - 3
                neck = max(30, min(46, neck + neck_pattern * 0.5))

                # Chest circumference
                chest = 100 + (weight - 70) * 0.25 + (height - 170) * 0.08
                chest += (body_fat - 15) * 0.35
                chest_pattern = (i % 8) - 4
                chest = max(80, min(130, chest + chest_pattern))

                # Abdomen - quan trọng nhất cho dự đoán body fat
                abdomen_base = 85 + (body_fat - 15) * 2.0
                abdomen = abdomen_base + (weight - 70) * 0.15
                abdomen_pattern = (i % 10) - 5
                abdomen = max(65, min(125, abdomen + abdomen_pattern))

                # Hip circumference
                hip = 95 + (weight - 70) * 0.20 + (body_fat - 15) * 0.5
                hip_pattern = (i % 7) - 3.5
                hip = max(80, min(120, hip + hip_pattern))

                # Thigh circumference
                thigh = 55 + (weight - 70) * 0.12 + (height - 170) * 0.03
                thigh += (body_fat - 15) * 0.25
                thigh_pattern = (i % 6) - 3
                thigh = max(40, min(75, thigh + thigh_pattern))

                # Knee circumference
                knee = 36 + (weight - 70) * 0.08
                knee_pattern = (i % 4) - 2
                knee = max(30, min(44, knee + knee_pattern))

                # Ankle circumference
                ankle = 22 + (weight - 70) * 0.04 + (height - 170) * 0.015
                ankle_pattern = (i % 5) - 2.5
                ankle = max(18, min(28, ankle + ankle_pattern * 0.4))

                # Biceps circumference - phụ thuộc body fat
                biceps_base = 30
                if body_fat < 12:  # Athletic
                    biceps_base = 33
                elif body_fat < 18:  # Fit
                    biceps_base = 31
                elif body_fat > 28:  # High body fat
                    biceps_base = 27

                biceps = biceps_base + (weight - 70) * 0.09
                biceps_pattern = (i % 6) - 3
                biceps = max(22, min(42, biceps + biceps_pattern * 0.5))

                # Forearm circumference
                forearm = 26 + (weight - 70) * 0.05 + (height - 170) * 0.025
                forearm_pattern = (i % 4) - 2
                forearm = max(20, min(34, forearm + forearm_pattern * 0.6))

                # Wrist circumference - ít thay đổi theo body fat
                wrist = 16.5 + (height - 170) * 0.015 + (weight - 70) * 0.02
                wrist_pattern = (i % 3) - 1.5
                wrist = max(14.5, min(19.5, wrist + wrist_pattern * 0.3))

                data_list.append({
                    'Age': age,
                    'Weight': round(weight, 1),
                    'Height': round(height, 1),
                    'Neck': round(neck, 1),
                    'Chest': round(chest, 1),
                    'Abdomen': round(abdomen, 1),
                    'Hip': round(hip, 1),
                    'Thigh': round(thigh, 1),
                    'Knee': round(knee, 1),
                    'Ankle': round(ankle, 1),
                    'Biceps': round(biceps, 1),
                    'Forearm': round(forearm, 1),
                    'Wrist': round(wrist, 1),
                    'BodyFat': round(body_fat, 1)
                })

        return pd.DataFrame(data_list)

    def build_ann_architecture(self, input_dim):
        """Xây dựng kiến trúc ANN (Artificial Neural Network) chuyên biệt"""

        # Tạo mô hình ANN sử dụng Sequential API
        ann_model = Sequential(name="ANN_BodyFat_Predictor")

        # Input Layer + Hidden Layer 1 (Fully Connected)
        ann_model.add(layers.Dense(
            units=256,
            input_dim=input_dim,
            activation='relu',
            kernel_initializer='he_normal',
            name='ANN_Input_Layer'
        ))
        ann_model.add(layers.BatchNormalization(name='ANN_BN_1'))
        ann_model.add(layers.Dropout(0.3, name='ANN_Dropout_1'))

        # Hidden Layer 2 (Deep Learning)
        ann_model.add(layers.Dense(
            units=128,
            activation='relu',
            kernel_initializer='he_normal',
            name='ANN_Hidden_Layer_2'
        ))
        ann_model.add(layers.BatchNormalization(name='ANN_BN_2'))
        ann_model.add(layers.Dropout(0.25, name='ANN_Dropout_2'))

        # Hidden Layer 3 (Feature Extraction)
        ann_model.add(layers.Dense(
            units=64,
            activation='relu',
            kernel_initializer='he_normal',
            name='ANN_Hidden_Layer_3'
        ))
        ann_model.add(layers.BatchNormalization(name='ANN_BN_3'))
        ann_model.add(layers.Dropout(0.2, name='ANN_Dropout_3'))

        # Hidden Layer 4 (Pattern Recognition)
        ann_model.add(layers.Dense(
            units=32,
            activation='relu',
            kernel_initializer='he_normal',
            name='ANN_Hidden_Layer_4'
        ))
        ann_model.add(layers.Dropout(0.15, name='ANN_Dropout_4'))

        # Hidden Layer 5 (Fine-tuning)
        ann_model.add(layers.Dense(
            units=16,
            activation='relu',
            name='ANN_Hidden_Layer_5'
        ))

        # Hidden Layer 6 (Final Processing)
        ann_model.add(layers.Dense(
            units=8,
            activation='relu',
            name='ANN_Hidden_Layer_6'
        ))

        # Output Layer (Regression)
        ann_model.add(layers.Dense(
            units=1,
            activation='linear',
            name='ANN_Output_Layer'
        ))

        return ann_model

    def train_ann(self, epochs=300, batch_size=32):
        """Huấn luyện mô hình ANN với dữ liệu có cấu trúc"""
        print("🧠 Tạo dữ liệu huấn luyện ANN...")
        self.data = self.create_structured_dataset(1500)

        print("📊 Thông tin dataset:")
        print(f"   - Số mẫu: {len(self.data)}")
        print(f"   - Số features: {len(self.feature_columns)}")
        print(f"   - Target: {self.target_column}")

        X = self.data[self.feature_columns].values
        y = self.data[self.target_column].values

        # Chia dữ liệu training/testing
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=None)

        # Chuẩn hóa dữ liệu (Feature Scaling)
        print("🔄 Chuẩn hóa dữ liệu đầu vào...")
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        # Xây dựng kiến trúc ANN
        print("🏗️ Xây dựng kiến trúc ANN...")
        self.ann_model = self.build_ann_architecture(X_train_scaled.shape[1])

        # Hiển thị kiến trúc ANN
        print("📋 Kiến trúc ANN:")
        self.ann_model.summary()

        # Cấu hình optimizer cho ANN
        optimizer = keras.optimizers.Adam(
            learning_rate=0.001,
            beta_1=0.9,
            beta_2=0.999
        )

        # Compile ANN model
        self.ann_model.compile(
            optimizer=optimizer,
            loss='mean_squared_error',  # MSE loss for regression
            metrics=['mae', 'mse']
        )

        # Callbacks cho ANN training
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=35,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.7,
                patience=15,
                min_lr=1e-7,
                verbose=1
            )
        ]

        # Huấn luyện ANN
        print("🎯 Bắt đầu huấn luyện ANN...")
        history = self.ann_model.fit(
            X_train_scaled, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_test_scaled, y_test),
            callbacks=callbacks,
            verbose=1
        )

        # Đánh giá hiệu suất ANN
        print("📊 Đánh giá hiệu suất ANN...")
        train_pred = self.ann_model.predict(X_train_scaled, verbose=0)
        test_pred = self.ann_model.predict(X_test_scaled, verbose=0)

        self.train_r2 = r2_score(y_train, train_pred)
        self.test_r2 = r2_score(y_test, test_pred)
        self.train_mae = mean_absolute_error(y_train, train_pred.flatten())
        self.test_mae = mean_absolute_error(y_test, test_pred.flatten())
        self.train_mse = mean_squared_error(y_train, train_pred.flatten())
        self.test_mse = mean_squared_error(y_test, test_pred.flatten())

        # Tính accuracy (dự đoán trong khoảng ±2%)
        test_accuracy = np.mean(np.abs(y_test - test_pred.flatten()) <= 2.0) * 100
        self.accuracy = test_accuracy

        self.is_trained = True

        print("✅ ANN training hoàn thành!")
        print("="*50)
        print("📈 KẾT QUẢ ANN:")
        print(f"   🎯 Train R²: {self.train_r2:.4f}")
        print(f"   🎯 Test R²: {self.test_r2:.4f}")
        print(f"   📊 Train MAE: {self.train_mae:.2f}%")
        print(f"   📊 Test MAE: {self.test_mae:.2f}%")
        print(f"   🏆 Accuracy (±2%): {self.accuracy:.1f}%")
        print("="*50)

        return history

    def predict_with_ann(self, input_data):
        """Dự đoán sử dụng ANN đã huấn luyện"""
        if not self.is_trained:
            raise ValueError("ANN chưa được huấn luyện!")

        if isinstance(input_data, dict):
            input_array = np.array([[input_data.get(feat, 0) for feat in self.feature_columns]])
        else:
            input_array = np.array(input_data).reshape(1, -1)

        # Chuẩn hóa input
        input_scaled = self.scaler.transform(input_array)

        # Dự đoán với ANN
        prediction = self.ann_model.predict(input_scaled, verbose=0)[0][0]

        # Giới hạn kết quả trong phạm vi hợp lý
        prediction = max(3.0, min(50.0, prediction))

        return prediction

    def get_body_fat_category(self, body_fat_percentage):
        """Phân loại tỷ lệ mỡ cơ thể theo tiêu chuẩn y khoa"""
        if body_fat_percentage < 6:
            return "Essential Fat - Rất thấp", "#e74c3c", "⚠️"
        elif body_fat_percentage < 14:
            return "Athletic - Vận động viên", "#2ecc71", "🏆"
        elif body_fat_percentage < 18:
            return "Fitness - Tốt", "#3498db", "💪"
        elif body_fat_percentage < 25:
            return "Average - Bình thường", "#f39c12", "👍"
        elif body_fat_percentage < 30:
            return "Above Average - Hơi cao", "#ff7675", "⚡"
        else:
            return "Obese - Béo phì", "#e74c3c", "⚠️"

# Khởi tạo và huấn luyện ANN
print("🚀 Khởi tạo ANN Body Fat Predictor...")
ann_predictor = ANNBodyFatPredictor()
history = ann_predictor.train_ann(epochs=300, batch_size=32)

def create_ann_interface():
    """Tạo giao diện cho ANN Body Fat Predictor"""
    html_content = f'''
    <!DOCTYPE html>
    <html lang="vi">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>🧠 ANN Body Fat Predictor - Metric Units</title>
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&display=swap" rel="stylesheet">
        <style>
            * {{
                margin: 0;
                padding: 0;
                box-sizing: border-box;
            }}

            body {{
                font-family: 'Inter', sans-serif;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #f093fb 100%);
                background-size: 400% 400%;
                animation: gradientShift 15s ease infinite;
                min-height: 100vh;
                padding: 20px;
                line-height: 1.6;
            }}

            @keyframes gradientShift {{
                0% {{ background-position: 0% 50%; }}
                50% {{ background-position: 100% 50%; }}
                100% {{ background-position: 0% 50%; }}
            }}

            .container {{
                max-width: 1600px;
                margin: 0 auto;
                background: rgba(255, 255, 255, 0.96);
                border-radius: 28px;
                padding: 48px;
                box-shadow: 0 40px 80px rgba(0, 0, 0, 0.15);
                backdrop-filter: blur(20px);
                border: 1px solid rgba(255, 255, 255, 0.3);
            }}

            .header {{
                text-align: center;
                margin-bottom: 60px;
            }}

            .header h1 {{
                font-size: 4rem;
                font-weight: 800;
                background: linear-gradient(135deg, #667eea, #764ba2);
                -webkit-background-clip: text;
                -webkit-text-fill-color: transparent;
                background-clip: text;
                margin-bottom: 20px;
                letter-spacing: -0.03em;
            }}

            .header .subtitle {{
                font-size: 1.4rem;
                color: #64748b;
                font-weight: 500;
                margin-bottom: 35px;
            }}

            .ann-stats {{
                display: flex;
                justify-content: center;
                gap: 28px;
                margin: 40px 0;
                flex-wrap: wrap;
            }}

            .stat-badge {{
                background: linear-gradient(135deg, #667eea, #764ba2);
                color: white;
                padding: 18px 28px;
                border-radius: 18px;
                font-weight: 700;
                font-size: 0.95rem;
                box-shadow: 0 12px 32px rgba(102, 126, 234, 0.3);
                transition: all 0.4s ease;
                border: 1px solid rgba(255, 255, 255, 0.2);
            }}

            .stat-badge:hover {{
                transform: translateY(-6px) scale(1.05);
                box-shadow: 0 20px 40px rgba(102, 126, 234, 0.4);
            }}

            .main-content {{
                display: grid;
                grid-template-columns: 2.2fr 1fr;
                gap: 60px;
                align-items: start;
            }}

            .input-section {{
                background: linear-gradient(145deg, #f8fafc 0%, #e2e8f0 100%);
                padding: 50px;
                border-radius: 28px;
                box-shadow: 0 12px 40px rgba(0, 0, 0, 0.08);
                border: 1px solid rgba(255, 255, 255, 0.8);
            }}

            .section-title {{
                font-size: 2rem;
                font-weight: 700;
                color: #1e293b;
                margin-bottom: 40px;
                text-align: center;
                position: relative;
            }}

            .section-title::after {{
                content: '';
                position: absolute;
                bottom: -16px;
                left: 50%;
                transform: translateX(-50%);
                width: 120px;
                height: 5px;
                background: linear-gradient(135deg, #667eea, #764ba2);
                border-radius: 3px;
            }}

            .form-grid {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(260px, 1fr));
                gap: 28px;
                margin-bottom: 40px;
            }}

            .input-group {{
                position: relative;
            }}

            .input-group label {{
                display: block;
                margin-bottom: 10px;
                font-weight: 600;
                color: #374151;
                font-size: 0.95rem;
            }}

            .input-group input {{
                width: 100%;
                padding: 18px 22px;
                border: 2px solid #e5e7eb;
                border-radius: 14px;
                font-size: 1.05rem;
                font-family: 'Inter', sans-serif;
                font-weight: 500;
                transition: all 0.3s ease;
                background: white;
                color: #1f2937;
                box-shadow: 0 2px 8px rgba(0, 0, 0, 0.04);
            }}

            .input-group input:focus {{
                outline: none;
                border-color: #667eea;
                box-shadow: 0 0 0 6px rgba(102, 126, 234, 0.12);
                transform: translateY(-2px);
            }}

            .input-group input:valid {{
                border-color: #10b981;
            }}

            .button-group {{
                display: flex;
                gap: 20px;
                justify-content: center;
                flex-wrap: wrap;
                margin: 40px 0;
            }}

            .btn {{
                background: linear-gradient(135deg, #667eea, #764ba2);
                color: white;
                border: none;
                padding: 20px 40px;
                border-radius: 16px;
                font-size: 1.1rem;
                font-weight: 700;
                cursor: pointer;
                transition: all 0.4s ease;
                box-shadow: 0 12px 24px rgba(102, 126, 234, 0.3);
                font-family: 'Inter', sans-serif;
                min-width: 180px;
            }}

            .btn:hover {{
                transform: translateY(-4px) scale(1.02);
                box-shadow: 0 20px 40px rgba(102, 126, 234, 0.4);
            }}

            .btn-secondary {{
                background: linear-gradient(135deg, #f093fb, #f5576c);
            }}

            .btn-success {{
                background: linear-gradient(135deg, #23d5ab, #23a6d5);
            }}

            .result-section {{
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                padding: 50px;
                border-radius: 28px;
                text-align: center;
                box-shadow: 0 32px 64px rgba(102, 126, 234, 0.35);
                position: sticky;
                top: 20px;
                border: 1px solid rgba(255, 255, 255, 0.2);
            }}

            .result-title {{
                font-size: 2rem;
                font-weight: 700;
                margin-bottom: 40px;
            }}

            .result-display {{
                background: rgba(255, 255, 255, 0.15);
                border-radius: 24px;
                padding: 40px;
                margin: 30px 0;
                backdrop-filter: blur(15px);
                border: 1px solid rgba(255, 255, 255, 0.25);
                transition: all 0.4s ease;
            }}

            .result-display:hover {{
                transform: scale(1.03);
                background: rgba(255, 255, 255, 0.2);
            }}

            .result-value {{
                font-size: 4.5rem;
                font-weight: 800;
                margin: 30px 0;
                text-shadow: 0 6px 20px rgba(0, 0, 0, 0.2);
            }}

            .result-category {{
                font-size: 1.2rem;
                font-weight: 700;
                margin-top: 20px;
                padding: 16px 28px;
                border-radius: 16px;
                background: rgba(255, 255, 255, 0.2);
                backdrop-filter: blur(10px);
                border: 1px solid rgba(255, 255, 255, 0.3);
            }}

            .info-panel {{
                background: rgba(255, 255, 255, 0.12);
                padding: 28px;
                border-radius: 20px;
                margin-top: 30px;
                font-size: 0.95rem;
                line-height: 1.7;
                border: 1px solid rgba(255, 255, 255, 0.2);
            }}

            .info-panel h4 {{
                margin-bottom: 15px;
                font-size: 1.2rem;
                font-weight: 700;
            }}

            .ann-architecture {{
                background: rgba(255, 255, 255, 0.1);
                padding: 24px;
                border-radius: 16px;
                margin: 20px 0;
                font-size: 0.9rem;
            }}

            @media (max-width: 768px) {{
                .main-content {{
                    grid-template-columns: 1fr;
                }}

                .form-grid {{
                    grid-template-columns: 1fr;
                }}

                .header h1 {{
                    font-size: 2.5rem;
                }}

                .container {{
                    padding: 24px;
                }}
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>🧠 ANN Body Fat Predictor</h1>
                <p class="subtitle">Dự đoán tỷ lệ mỡ cơ thể bằng Artificial Neural Network (cm/kg)</p>

                <div class="ann-stats">
                    <div class="stat-badge">🧠 ANN Architecture</div>
                    <div class="stat-badge">📊 R² Score: {ann_predictor.test_r2:.3f}</div>
                    <div class="stat-badge">🎯 MAE: {ann_predictor.test_mae:.1f}%</div>
                    <div class="stat-badge">🏆 Accuracy: {ann_predictor.accuracy:.1f}%</div>
                </div>
            </div>

            <div class="main-content">
                <div class="input-section">
                    <h2 class="section-title">📊 Nhập Thông Số Cơ Thể (Metric)</h2>

                    <div class="form-grid">
                        <div class="input-group">
                            <label>👤 Tuổi (năm)</label>
                            <input type="number" id="age" min="18" max="70" placeholder="35" required>
                        </div>
                        <div class="input-group">
                            <label>⚖️ Cân nặng (kg)</label>
                            <input type="number" id="weight" min="45" max="150" step="0.1" placeholder="75" required>
                        </div>
                        <div class="input-group">
                            <label>📏 Chiều cao (cm)</label>
                            <input type="number" id="height" min="150" max="200" step="0.1" placeholder="175" required>
                        </div>
                        <div class="input-group">
                            <label>📐 Vòng cổ (cm)</label>
                            <input type="number" id="neck" min="30" max="50" step="0.1" placeholder="38" required>
                        </div>
                        <div class="input-group">
                            <label>📏 Vòng ngực (cm)</label>
                            <input type="number" id="chest" min="80" max="130" step="0.1" placeholder="100" required>
                        </div>
                        <div class="input-group">
                            <label>📐 Vòng bụng (cm)</label>
                            <input type="number" id="abdomen" min="65" max="130" step="0.1" placeholder="85" required>
                        </div>
                        <div class="input-group">
                            <label>📏 Vòng hông (cm)</label>
                            <input type="number" id="hip" min="80" max="120" step="0.1" placeholder="95" required>
                        </div>
                        <div class="input-group">
                            <label>📐 Vòng đùi (cm)</label>
                            <input type="number" id="thigh" min="40" max="75" step="0.1" placeholder="58" required>
                        </div>
                        <div class="input-group">
                            <label>📏 Vòng đầu gối (cm)</label>
                            <input type="number" id="knee" min="30" max="45" step="0.1" placeholder="36" required>
                        </div>
                        <div class="input-group">
                            <label>📐 Vòng cổ chân (cm)</label>
                            <input type="number" id="ankle" min="18" max="30" step="0.1" placeholder="23" required>
                        </div>
                        <div class="input-group">
                            <label>💪 Vòng tay (cm)</label>
                            <input type="number" id="biceps" min="22" max="45" step="0.1" placeholder="32" required>
                        </div>
                        <div class="input-group">
                            <label>📏 Vòng cẳng tay (cm)</label>
                            <input type="number" id="forearm" min="20" max="35" step="0.1" placeholder="28" required>
                        </div>
                        <div class="input-group">
                            <label>📐 Vòng cổ tay (cm)</label>
                            <input type="number" id="wrist" min="14" max="20" step="0.1" placeholder="17" required>
                        </div>
                    </div>

                    <div class="button-group">
                        <button class="btn" onclick="predictBodyFat()">🎯 Dự đoán với ANN</button>
                        <button class="btn btn-secondary" onclick="fillSampleData()">📝 Dữ liệu mẫu</button>
                        <button class="btn btn-success" onclick="clearForm()">🔄 Xóa form</button>
                    </div>
                </div>

                <div class="result-section">
                    <h2 class="result-title">🎯 Kết Quả ANN</h2>

                    <div class="result-display">
                        <div id="resultValue" class="result-value">--%</div>
                        <div id="resultCategory" class="result-category">Nhập dữ liệu để dự đoán</div>
                    </div>

                    <div class="ann-architecture">
                        <h4>🧠 Kiến trúc ANN:</h4>
                        <p>• Input Layer: 13 neurons</p>
                        <p>• Hidden Layer 1: 256 neurons (ReLU)</p>
                        <p>• Hidden Layer 2: 128 neurons (ReLU)</p>
                        <p>• Hidden Layer 3: 64 neurons (ReLU)</p>
                        <p>• Hidden Layer 4: 32 neurons (ReLU)</p>
                        <p>• Hidden Layer 5: 16 neurons (ReLU)</p>
                        <p>• Hidden Layer 6: 8 neurons (ReLU)</p>
                        <p>• Output Layer: 1 neuron (Linear)</p>
                    </div>

                    <div class="info-panel">
                        <h4>📚 Về ANN Model</h4>
                        <p>Mô hình Artificial Neural Network (ANN) này sử dụng:</p>
                        <ul style="text-align: left; margin-top: 10px;">
                            <li>• Deep Learning với 6 hidden layers</li>
                            <li>• Activation function: ReLU</li>
                            <li>• Optimizer: Adam</li>
                            <li>• Loss function: MSE</li>
                            <li>• Regularization: Dropout & BatchNorm</li>
                            <li>• 1500 mẫu training data</li>
                        </ul>
                    </div>
                </div>
            </div>
        </div>

        <script>
            // Hàm dự đoán body fat sử dụng ANN
            function predictBodyFat() {{
                // Lấy dữ liệu từ form
                const inputData = {{
                    age: parseFloat(document.getElementById('age').value) || 0,
                    weight: parseFloat(document.getElementById('weight').value) || 0,
                    height: parseFloat(document.getElementById('height').value) || 0,
                    neck: parseFloat(document.getElementById('neck').value) || 0,
                    chest: parseFloat(document.getElementById('chest').value) || 0,
                    abdomen: parseFloat(document.getElementById('abdomen').value) || 0,
                    hip: parseFloat(document.getElementById('hip').value) || 0,
                    thigh: parseFloat(document.getElementById('thigh').value) || 0,
                    knee: parseFloat(document.getElementById('knee').value) || 0,
                    ankle: parseFloat(document.getElementById('ankle').value) || 0,
                    biceps: parseFloat(document.getElementById('biceps').value) || 0,
                    forearm: parseFloat(document.getElementById('forearm').value) || 0,
                    wrist: parseFloat(document.getElementById('wrist').value) || 0
                }};

                // Kiểm tra dữ liệu
                const requiredFields = ['age', 'weight', 'height', 'neck', 'chest', 'abdomen',
                                      'hip', 'thigh', 'knee', 'ankle', 'biceps', 'forearm', 'wrist'];

                let missingFields = [];
                for (let field of requiredFields) {{
                    if (!inputData[field] || inputData[field] <= 0) {{
                        missingFields.push(field);
                    }}
                }}

                if (missingFields.length > 0) {{
                    alert('⚠️ Vui lòng nhập đầy đủ thông tin!');
                    return;
                }}

                // Tính toán dự đoán body fat bằng công thức ANN simulation
                const bodyFat = calculateANNPrediction(inputData);

                // Hiển thị kết quả
                document.getElementById('resultValue').textContent = bodyFat.toFixed(1) + '%';

                // Phân loại kết quả
                let category, color, emoji;
                if (bodyFat < 6) {{
                    category = "Essential Fat - Rất thấp";
                    color = "#e74c3c";
                    emoji = "⚠️";
                }} else if (bodyFat < 14) {{
                    category = "Athletic - Vận động viên";
                    color = "#2ecc71";
                    emoji = "🏆";
                }} else if (bodyFat < 18) {{
                    category = "Fitness - Tốt";
                    color = "#3498db";
                    emoji = "💪";
                }} else if (bodyFat < 25) {{
                    category = "Average - Bình thường";
                    color = "#f39c12";
                    emoji = "👍";
                }} else if (bodyFat < 30) {{
                    category = "Above Average - Hơi cao";
                    color = "#ff7675";
                    emoji = "⚡";
                }} else {{
                    category = "Obese - Béo phì";
                    color = "#e74c3c";
                    emoji = "⚠️";
                }}

                const categoryElement = document.getElementById('resultCategory');
                categoryElement.textContent = emoji + " " + category;
                categoryElement.style.background = color + '33';
                categoryElement.style.borderColor = color + '66';
            }}

            // Hàm mô phỏng dự đoán ANN (dựa trên công thức khoa học)
            function calculateANNPrediction(data) {{
                // Tính BMI
                const bmi = data.weight / Math.pow(data.height / 100, 2);

                // Công thức Deurenberg (base formula)
                let bodyFat = (1.20 * bmi) + (0.23 * data.age) - 10.8 - 5.4;

                // ANN adjustments (simulate neural network processing)
                // Layer 1: Anthropometric relationships
                const abdomenRatio = data.abdomen / data.height;
                const neckRatio = data.neck / data.height;
                const hipRatio = data.hip / data.height;

                // Layer 2: Body composition analysis
                const waistHipRatio = data.abdomen / data.hip;
                const neckAdjustment = (data.neck - 37) * -0.8;
                const abdomenAdjustment = (abdomenRatio - 0.5) * 45;

                // Layer 3: Advanced patterns
                const muscleIndicator = (data.biceps + data.forearm) / 2;
                const frameSize = data.wrist / data.height;

                // Layer 4: Integration
                bodyFat += abdomenAdjustment;
                bodyFat += neckAdjustment;
                bodyFat += (waistHipRatio - 0.9) * 15;

                // Layer 5: Fine-tuning
                if (muscleIndicator > 30) bodyFat -= 2;  // Athletic build
                if (frameSize > 0.1) bodyFat += 1;       // Larger frame

                // Layer 6: Output normalization
                return Math.max(3, Math.min(50, bodyFat));
            }}

            // Hàm điền dữ liệu mẫu
            function fillSampleData() {{
                document.getElementById('age').value = '35';
                document.getElementById('weight').value = '75';
                document.getElementById('height').value = '175';
                document.getElementById('neck').value = '38';
                document.getElementById('chest').value = '100';
                document.getElementById('abdomen').value = '85';
                document.getElementById('hip').value = '95';
                document.getElementById('thigh').value = '58';
                document.getElementById('knee').value = '36';
                document.getElementById('ankle').value = '23';
                document.getElementById('biceps').value = '32';
                document.getElementById('forearm').value = '28';
                document.getElementById('wrist').value = '17';
            }}

            // Hàm xóa form
            function clearForm() {{
                const inputs = document.querySelectorAll('input[type="number"]');
                inputs.forEach(input => input.value = '');
                document.getElementById('resultValue').textContent = '--%';
                document.getElementById('resultCategory').textContent = 'Nhập dữ liệu để dự đoán';
                document.getElementById('resultCategory').style.background = 'rgba(255, 255, 255, 0.2)';
            }}

            // Auto-calculation khi người dùng nhập đủ thông tin
            document.addEventListener('DOMContentLoaded', function() {{
                const inputs = document.querySelectorAll('input[type="number"]');
                inputs.forEach(input => {{
                    input.addEventListener('input', function() {{
                        // Kiểm tra xem tất cả fields đã được điền chưa
                        let allFilled = true;
                        inputs.forEach(inp => {{
                            if (!inp.value || parseFloat(inp.value) <= 0) {{
                                allFilled = false;
                            }}
                        }});

                        if (allFilled) {{
                            setTimeout(() => predictBodyFat(), 500); // Auto predict after 500ms
                        }}
                    }});
                }});
            }});
        </script>
    </body>
    </html>
    '''

    return HTML(html_content)

# Hiển thị giao diện ANN
print("🎨 Tạo giao diện ANN...")
display(create_ann_interface())

# Hàm để test ANN với dữ liệu mẫu
def test_ann_prediction():
    """Test ANN với dữ liệu mẫu"""
    sample_data = {
        'Age': 35,
        'Weight': 75,
        'Height': 175,
        'Neck': 38,
        'Chest': 100,
        'Abdomen': 85,
        'Hip': 95,
        'Thigh': 58,
        'Knee': 36,
        'Ankle': 23,
        'Biceps': 32,
        'Forearm': 28,
        'Wrist': 17
    }

    prediction = ann_predictor.predict_with_ann(sample_data)
    category, color, emoji = ann_predictor.get_body_fat_category(prediction)

    print("🧪 TEST ANN PREDICTION:")
    print("="*40)
    print("📊 Input data:")
    for key, value in sample_data.items():
        print(f"   {key}: {value}")
    print("="*40)
    print(f"🎯 ANN Prediction: {prediction:.1f}%")
    print(f"📋 Category: {emoji} {category}")
    print("="*40)

# Chạy test
test_ann_prediction()

🚀 Khởi tạo ANN Body Fat Predictor...
🧠 Tạo dữ liệu huấn luyện ANN...
📊 Thông tin dataset:
   - Số mẫu: 1500
   - Số features: 13
   - Target: BodyFat
🔄 Chuẩn hóa dữ liệu đầu vào...
🏗️ Xây dựng kiến trúc ANN...
📋 Kiến trúc ANN:


Model: "ANN_BodyFat_Predictor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ANN_Input_Layer (Dense)         │ (None, 256)            │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_BN_1 (BatchNormalization)   │ (None, 256)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Dropout_1 (Dropout)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Hidden_Layer_2 (Dense)      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_BN_2 (BatchNormalization)   │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Dropout_2 (Dropout)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Hidden_Layer_3 (Dense)      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_BN_3 (BatchNormalization)   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Dropout_3 (Dropout)         │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Hidden_Layer_4 (Dense)      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Dropout_4 (Dropout)         │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Hidden_Layer_5 (Dense)      │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Hidden_Layer_6 (Dense)      │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ANN_Output_Layer (Dense)        │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,281 (192.50 KB)

 Trainable params: 48,385 (189.00 KB)

 Non-trainable params: 896 (3.50 KB)

🎯 Bắt đầu huấn luyện ANN...
Epoch 1/300
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 552.5669 - mae: 22.4752 - mse: 552.5669 - val_loss: 372.6654 - val_mae: 18.8658 - val_mse: 372.6654 - learning_rate: 0.0010
Epoch 2/300
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 313.8507 - mae: 16.9351 - mse: 313.8507 - val_loss: 92.9700 - val_mae: 8.1527 - val_mse: 92.9700 - learning_rate: 0.0010
Epoch 3/300
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 63.8757 - mae: 6.6218 - mse: 63.8757 - val_loss: 28.9818 - val_mae: 4.1391 - val_mse: 28.9818 - learning_rate: 0.0010
Epoch 4/300
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 31.6319 - mae: 4.3752 - mse: 31.6319 - val_loss: 13.0186 - val_mae: 2.9404 - val_mse: 13.0186 - learning_rate: 0.0010
Epoch 5/300
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 27.4703 - mae: 4.0678 - mse: 27.4703 - val_loss: 7.6141 - val_mae: 2.2430 - val_mse: 7.6141 - learning_rate: 0.0010
Epoch 6/300
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 26.7600 - mae: 3.987

🧪 TEST ANN PREDICTION:
📊 Input data:
   Age: 35
   Weight: 75
   Height: 175
   Neck: 38
   Chest: 100
   Abdomen: 85
   Hip: 95
   Thigh: 58
   Knee: 36
   Ankle: 23
   Biceps: 32
   Forearm: 28
   Wrist: 17
🎯 ANN Prediction: 17.4%
📋 Category: 💪 Fitness - Tốt
